Below code is for regression

In [ ]:
import socket
import numpy as np
import struct
import ssl
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_diabetes


dataset = load_diabetes()
X = dataset.data
y = dataset.target


X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X=X_train
Y=Y_train
Y=Y.reshape(X.shape[0],1)
def recv_full(ssl_socket, num_bytes):
    
    buffer = b''
    while len(buffer) < num_bytes:
        packet = ssl_socket.recv(num_bytes - len(buffer))
        if not packet:
            break
        buffer += packet
    return buffer


host = socket.gethostbyname(socket.gethostname())
port = 5060


context = ssl.create_default_context()
context.check_hostname = False
context.verify_mode = ssl.CERT_NONE

client_server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
ssl_socket = context.wrap_socket(client_server)
ssl_socket.connect((host, port))
#password
while True:
    password=input('enter password:')
    ssl_socket.send(password.encode('utf-8'))
    con=ssl_socket.recv().decode('utf-8')
    if con=='ok':
        break
    else:
        continue
choice = input('To perform regression enter 1, to perform classification enter 2, to perform multi-class classification enter 3: ')
neurons = int(input('Enter the number of neurons you want to train the model on: '))
learning_rate = float(input('Enter the learning rate of the model: '))
iter = int(input('Enter the number of iterations of the model: '))


# Send X
array_shapeX = ','.join(map(str, X.shape))
shape_lengthX = str(len(array_shapeX)).ljust(10)
ssl_socket.send(shape_lengthX.encode('utf-8'))
ssl_socket.send(array_shapeX.encode('utf-8'))
ssl_socket.send(X.tobytes())
# Send Y
array_shapeY = ','.join(map(str, Y.shape))
shape_lengthY = str(len(array_shapeY)).ljust(10)
ssl_socket.send(shape_lengthY.encode('utf-8'))
ssl_socket.send(array_shapeY.encode('utf-8'))
ssl_socket.send(Y.tobytes())
# Send X_test
array_shapeX_test = ','.join(map(str, X_test.shape))
shape_lengthX_test = str(len(array_shapeX_test)).ljust(10)
ssl_socket.send(shape_lengthX_test.encode('utf-8'))
ssl_socket.send(array_shapeX_test.encode('utf-8'))
ssl_socket.send(X_test.tobytes())

# Send model parameters
ssl_socket.send(choice.encode('utf-8'))
ssl_socket.send(struct.pack('i', neurons))
ssl_socket.send(struct.pack('d', learning_rate))
ssl_socket.send(struct.pack('i', iter))

# Receiving A2
shape_length_header = ssl_socket.recv(10).decode('utf-8').strip()
shape_length = int(shape_length_header)
shape_array = ssl_socket.recv(shape_length).decode('utf-8')
shape = tuple(map(int, shape_array.split(',')))



num_bytes_A2 = np.prod(shape) * 8
array_bits = recv_full(ssl_socket, num_bytes_A2)


A2 = np.frombuffer(array_bits, dtype=np.float64).reshape(shape)
mape = np.mean(np.abs((Y_test - A2) / Y_test)) * 100
print('The accuracy of the model is ', round(mape, 2), '%')

ssl_socket.close()


W1 from server  127.0.1.1  is  [[-0.0011588  -0.01442195  0.00742209]
 [ 0.00725746  0.00269906 -0.00327643]
 [ 0.00871391  0.02052385 -0.00048661]
 ...
 [-0.00281239 -0.00490649  0.00544533]
 [ 0.01910989 -0.02259181 -0.00567987]
 [-0.01113889 -0.00715909 -0.00037738]]
Received W2 with shape: (1, 1000) and values: [[ 1.75631823e-02  2.47248147e-02  7.32966688e-03 -3.71812330e-03
  -1.19237968e-02  9.90520921e-04  3.35111181e-03  9.17201431e-04
  -4.18843129e-03  4.64573921e-03 -7.18940528e-03  3.58132119e-03
  -5.12910450e-03  1.44755024e-03  4.89706588e-03 -3.19272018e-03
   5.90669617e-03 -3.08799415e-03  5.12598145e-03 -1.64315365e-04
   4.62871962e-03 -1.01754301e-02  7.06403146e-04  5.64457786e-03
   1.18109584e-02 -1.05855177e-02  2.11566699e-03  2.91353662e-03
  -4.03074222e-03  5.82671550e-03  1.24047296e-02  3.10844923e-03
  -3.10991998e-03  2.08151298e-02  5.81321104e-03 -1.76574364e-03
   4.41006905e-03 -8.58370588e-03  1.42305635e-02  9.33322857e-03
   1.80629010e-03  7.37

Below code is for classification

In [ ]:
import socket
import numpy as np
import struct
import ssl
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn.datasets
dataset=sklearn.datasets.load_breast_cancer()
data=pd.DataFrame(dataset.data,columns=dataset.feature_names)
X=data.to_numpy()
data_label=pd.DataFrame(dataset.target)
Y=data_label.to_numpy()
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.20,random_state=42)
X=X_train
Y=Y_train

def recv_full(ssl_socket, num_bytes):
    
    buffer = b''
    while len(buffer) < num_bytes:
        packet = ssl_socket.recv(num_bytes - len(buffer))
        if not packet:
            break
        buffer += packet
    return buffer


host = socket.gethostbyname(socket.gethostname())
port = 5060


context = ssl.create_default_context()
context.check_hostname = False
context.verify_mode = ssl.CERT_NONE

client_server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
ssl_socket = context.wrap_socket(client_server)
ssl_socket.connect((host, port))
#password
while True:
    password=input('enter password:')
    ssl_socket.send(password.encode('utf-8'))
    con=ssl_socket.recv().decode('utf-8')
    if con=='ok':
        break
    else:
        continue
choice = input('To perform regression enter 1, to perform classification enter 2, to perform multi-class classification enter 3: ')
neurons = int(input('Enter the number of neurons you want to train the model on: '))
learning_rate = float(input('Enter the learning rate of the model: '))
iter = int(input('Enter the number of iterations of the model: '))


# Send X
array_shapeX = ','.join(map(str, X.shape))
shape_lengthX = str(len(array_shapeX)).ljust(10)
ssl_socket.send(shape_lengthX.encode('utf-8'))
ssl_socket.send(array_shapeX.encode('utf-8'))
ssl_socket.send(X.tobytes())
# Send Y
array_shapeY = ','.join(map(str, Y.shape))
shape_lengthY = str(len(array_shapeY)).ljust(10)
ssl_socket.send(shape_lengthY.encode('utf-8'))
ssl_socket.send(array_shapeY.encode('utf-8'))
ssl_socket.send(Y.tobytes())
# Send X_test
array_shapeX_test = ','.join(map(str, X_test.shape))
shape_lengthX_test = str(len(array_shapeX_test)).ljust(10)
ssl_socket.send(shape_lengthX_test.encode('utf-8'))
ssl_socket.send(array_shapeX_test.encode('utf-8'))
ssl_socket.send(X_test.tobytes())

# Send model parameters
ssl_socket.send(choice.encode('utf-8'))
ssl_socket.send(struct.pack('i', neurons))
ssl_socket.send(struct.pack('d', learning_rate))
ssl_socket.send(struct.pack('i', iter))

# Receiving A2
shape_length_header = ssl_socket.recv(10).decode('utf-8').strip()
shape_length = int(shape_length_header)
shape_array = ssl_socket.recv(shape_length).decode('utf-8')
shape = tuple(map(int, shape_array.split(',')))



num_bytes_A2 = np.prod(shape) * 8
array_bits = recv_full(ssl_socket, num_bytes_A2)


A2 = np.frombuffer(array_bits, dtype=np.float64).reshape(shape).copy()

for i in range(0,A2.shape[0]):
    if A2[i]<0.5:
        A2[i]=0
    else:
        A2[i]=1
correct=0
for i in range(0,A2.shape[0]):
    if(A2[i]==Y_test[i]):
        correct+=1


print('The accuracy of the model is ',round((correct/A2.shape[0])*100,2),'%')

ssl_socket.close()


W1 from server  127.0.1.1  is  [[0.55257987 0.72323916 0.48667007]
 [0.44298948 0.09562228 0.55107484]
 [0.25915039 0.04353426 0.73289904]
 ...
 [0.72777609 0.06252268 0.60753962]
 [0.77317693 0.30983396 0.75999932]
 [0.66216803 0.45844387 0.46448169]]
Received W2 with shape: (1, 1000) and values: [[ 2.39377268e-01  2.59067754e-01  2.13129098e-01 -1.29453604e-01
   4.07501937e-01  1.74998153e-01  1.93405423e-01  3.04678054e-02
  -9.47687008e-02 -3.33424991e-01  3.82195889e-01 -1.48874877e-01
   8.93577961e-01  3.96391281e-01 -4.03140551e-02  2.61086266e-01
  -1.45284467e-01 -4.56140161e-01  4.01257326e-01 -3.79018416e-01
  -3.71799865e-02  6.55899107e-01  3.90755645e-02 -1.30931030e-01
   5.92132150e-02  1.77255090e-01  5.49748436e-02  6.29984713e-01
   3.09574818e-01  3.49776711e-01  6.39971395e-01  4.35874328e-01
  -2.82632963e-01 -1.12477690e-01  1.10469170e-01  1.75918425e-01
  -2.55114019e-01 -3.83949927e-01  3.26938634e-01  1.02863522e-01
   2.72990925e-01  8.01224656e-01 -1.1698

Below code is for multi-classification

In [ ]:
import socket
import numpy as np
import struct
import ssl
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris


iris = load_iris()


X = pd.DataFrame(data=iris.data, columns=iris.feature_names)


Y = pd.get_dummies(iris.target, prefix='class')


Y = Y.astype(int)


X_numpy = X.to_numpy()
Y_numpy = Y.to_numpy()
X,X_test,Y,Y_test=train_test_split(X_numpy,Y_numpy,test_size=0.20,random_state=42)

def recv_full(ssl_socket, num_bytes):
    
    buffer = b''
    while len(buffer) < num_bytes:
        packet = ssl_socket.recv(num_bytes - len(buffer))
        if not packet:
            break
        buffer += packet
    return buffer


host = socket.gethostbyname(socket.gethostname())
port = 5060

context = ssl.create_default_context()
context.check_hostname = False
context.verify_mode = ssl.CERT_NONE

client_server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
ssl_socket = context.wrap_socket(client_server)
ssl_socket.connect((host, port))
#password
while True:
    password=input('enter password:')
    ssl_socket.send(password.encode('utf-8'))
    con=ssl_socket.recv().decode('utf-8')
    if con=='ok':
        break
    else:
        continue

choice = input('To perform regression enter 1, to perform classification enter 2, to perform multi-class classification enter 3: ')
neurons = int(input('Enter the number of neurons you want to train the model on: '))
learning_rate = float(input('Enter the learning rate of the model: '))
iter = int(input('Enter the number of iterations of the model: '))


# Send X
array_shapeX = ','.join(map(str, X.shape))
shape_lengthX = str(len(array_shapeX)).ljust(10)
ssl_socket.send(shape_lengthX.encode('utf-8'))
ssl_socket.send(array_shapeX.encode('utf-8'))
ssl_socket.send(X.tobytes())
# Send Y
array_shapeY = ','.join(map(str, Y.shape))
shape_lengthY = str(len(array_shapeY)).ljust(10)
ssl_socket.send(shape_lengthY.encode('utf-8'))
ssl_socket.send(array_shapeY.encode('utf-8'))
ssl_socket.send(Y.tobytes())
# Send X_test
array_shapeX_test = ','.join(map(str, X_test.shape))
shape_lengthX_test = str(len(array_shapeX_test)).ljust(10)
ssl_socket.send(shape_lengthX_test.encode('utf-8'))
ssl_socket.send(array_shapeX_test.encode('utf-8'))
ssl_socket.send(X_test.tobytes())

# Send model parameters
ssl_socket.send(choice.encode('utf-8'))
ssl_socket.send(struct.pack('i', neurons))
ssl_socket.send(struct.pack('d', learning_rate))
ssl_socket.send(struct.pack('i', iter))

# Receiving A2
shape_length_header = ssl_socket.recv(10).decode('utf-8').strip()
shape_length = int(shape_length_header)
shape_array = ssl_socket.recv(shape_length).decode('utf-8')
shape = tuple(map(int, shape_array.split(',')))



num_bytes_A2 = np.prod(shape) * 8
array_bits = recv_full(ssl_socket, num_bytes_A2)


A2 = np.frombuffer(array_bits, dtype=np.float64).reshape(shape).copy()
max_indices = np.argmax(A2, axis=1)


A2 = np.zeros_like(A2)

A2[np.arange(A2.shape[0]), max_indices] = 1

print('The accuracy of the model is ',round(np.mean(A2==Y_test)*100,2),'%')

ssl_socket.close()


W1 from server  127.0.1.1  is  [[-0.00513198 -0.01937517 -0.00028645 ...  0.01963249  0.00841045
   0.01337249]
 [ 0.00337271 -0.00669062 -0.01289501 ...  0.00677115  0.00905458
  -0.00033676]
 [ 0.00617059  0.00678028 -0.00384094 ...  0.00117451  0.00090615
   0.00902448]
 ...
 [-0.01550683  0.0177686   0.00372411 ...  0.0086573  -0.00407288
   0.00432053]
 [ 0.00863093 -0.00123659 -0.02364181 ...  0.00992948  0.00346659
   0.007534  ]
 [-0.00768386 -0.0020744   0.00302305 ... -0.00493942  0.00693374
   0.00333937]]
Received W2 with shape: (4, 1000) and values: [[-0.01138224  0.00242    -0.01770768 ...  0.00096005  0.00061863
   0.00813555]
 [ 0.00171708  0.01466244 -0.02902418 ... -0.00011361  0.01777174
  -0.00403471]
 [ 0.02260469 -0.01273291 -0.00504084 ... -0.01294235 -0.01042113
  -0.01723221]
 [-0.00677647 -0.01015841  0.00177833 ... -0.00615431  0.0081808
   0.00536854]]
Received B1 with shape: (1000, 1) and values: [[ 6.25314145e-06]
 [ 1.00039920e-06]
 [-1.42034623e-05]
 [ 6